# PROCESAMIENTO

### 01 Importación de paquetes a usar e iniciación de H2O

In [ ]:
import h2o
from h2o.estimators import H2OTargetEncoderEstimator
import xgboost
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import PowerTransformer, RobustScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score 
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_val_score, train_test_split
import time
h2o.init()

### 02 Procesamiento Dataset final

In [ ]:
go=pd.read_csv("../input/datos-prefinal-csv/datos_prefinal.csv")
go2=pd.read_csv("../input/datos-prefinal-csv/datos_prefinal.csv")
go2=go2.drop("Unnamed: 0",axis=1)

# Asignamos NAN a los valores futuros (mayores al 2022-05-01)
def dateNA(dataframe,column_name):
    mask=pd.to_datetime(dataframe[column_name])>pd.Timestamp("2022-05-01")
    dataframe.loc[mask,column_name]=np.nan
       
dateNA(go2,"FECINGRESOCLIENTE")
dateNA(go2,"FECACTIVACIONCONTRATO")
dateNA(go2,"LANZAMIENTO") 

# Transformamos las fechas para que reflejen la antigüedad en meses
def change_to_antiquity_month(dataframe, column_name):
    dataframe[column_name]=round((pd.Timestamp.today()-pd.to_datetime(dataframe[column_name])) / np.timedelta64(1, 'M'),1)
    
change_to_antiquity_month(go2,"FECINGRESOCLIENTE")
change_to_antiquity_month(go2,"FECACTIVACIONCONTRATO")
change_to_antiquity_month(go2,"LANZAMIENTO")

# Expresamos la variable Tipo_ADQ en dos columnas dummies
a=pd.get_dummies(go2["TIPO_ADQ"])
a.rename(columns={1:"Tipo_1",2:"Tipo_2"},inplace=True)
go2=pd.concat([go2, a], axis=1)
go2.drop("TIPO_ADQ",axis=1,inplace=True)

# Generamos una variable tráfico_suma como la suma de las variables tráficos del 3 al 9
go2["traf_sum"]=0
for i in range(3,10):
    go2["traf_sum"]+=go2["trafico_app_"+str(i)]
    
# Una vez sumados los dropeamos
for i in range(3,10):
  go3=go3.drop("trafico_app_"+str(i),axis=1)
    
# Seleccionamos las variables a escalar   
escalar=["FECINGRESOCLIENTE","FECACTIVACIONCONTRATO","LANZAMIENTO","VCHMESADENDA","trafico_app_1","trafico_app_2","traf_sum","trafico_total","GIGAS","MINUTOS","MENSAJES","mins_flujo_1","mins_flujo_2","VCHPENALIDAD"]
transformer = RobustScaler(quantile_range=(25.0, 75.0))
go2[escalar]=transformer.fit_transform(go2[escalar])


In [ ]:
# Expresamos NAN como -999 (para identificarlos con el algoritmo)
go3=pd.DataFrame()
for column in go2.columns.tolist():
    go3[column] = go2[column].fillna(-999)

# Expresamos el número de periodo como 1-4
for i in range(0,5):
  go3["NUMPERIODO"].replace(202200+i,i,inplace=True)

# Separamos los datos de entrenamiento y validación:
Modelo=go3[go3["NUMPERIODO"]!=4]
Prueba=go3[go3["NUMPERIODO"]==4]

# Dropeamos el número de periodo
Modelo=Modelo.drop("NUMPERIODO",axis=1)
Prueba=Prueba.drop("NUMPERIODO",axis=1)

# Trasformamos el dataframe de pandas a un dataframe de h2o, para aplicar con ese paquete un target (mean) encoding 
hf = h2o.H2OFrame(Modelo)
hf["TARGET"] = hf["TARGET"].asfactor()

rf = h2o.H2OFrame(Prueba)
rf["TARGET"] = rf["TARGET"].asfactor()

### 03 Mean Encoding

In [ ]:
# Seleccionamos las variables cualitativas en las cuales usaremos el mean encoding 
from h2o.estimators import H2OTargetEncoderEstimator
encoded_columns = ["MARCA","MODELO","NUEVA_GAMMA","OS","DEVICE_TYPE","GIRO","SUBGIRO"]

hf["fold"] = hf.kfold_column(n_folds=5, seed=25)

# Para el mean encoding usamos un k-fold de 5 divisiones e introducimos ruido blanco para prevenir overfit
te = H2OTargetEncoderEstimator(
    data_leakage_handling = "k_fold",
    fold_column = "fold",
    noise = 0.02,
    blending = True,
    inflection_point = 10,
    smoothing = 20)

te.train(x = encoded_columns, y = "TARGET",
    training_frame = hf)

# Transformamos el dataset de prueba y testeo (este último, sin ruido blanco)
train_te = te.transform(frame = hf)
test_te = te.transform(frame = rf, noise = 0.0)

### 04 Modelo y Prueba

In [ ]:
# Transformamos a pandas y dropeamos las columnas originales:
Modelo = train_te.as_data_frame()
Prueba = test_te.as_data_frame()

Modelo=Modelo.drop(encoded_columns,axis=1)
Prueba=Prueba.drop(encoded_columns,axis=1)

# Seleccionamos unicamente las variables de tipo float e integer
Modelo=Modelo.select_dtypes(include=["float64","int"]).drop("fold",axis=1)
Prueba=Prueba.select_dtypes(include=["float64","int"])

### 05 Búsqueda de hiperparámetros, ajuste del modelo y exportacion a Kaggle